In [39]:
from datasets.hand_pose_dataset import HandPoseDataset
from os.path import join, isdir
from os import listdir
import yaml
import pandas as pd

In [40]:
checkpoints_path = join("checkpoints", "ablation")
subject_ids = HandPoseDataset._get_subject_ids()
data = {}
metrics = []
for checkpoint_name in sorted(listdir(checkpoints_path)):
    # check if the checkpoint has complete experiments
    subjects_in_checkpoint = []
    for folder in listdir(join(checkpoints_path, checkpoint_name)):
        if folder in subject_ids and isdir(join(checkpoints_path, checkpoint_name, folder)) and "metrics.yaml" in listdir(join(checkpoints_path, checkpoint_name, folder)):
            subjects_in_checkpoint.append(folder)
    # skips empty checkpoints
    if len(subjects_in_checkpoint) <= 1:
        continue
    # loads the cfg used
    with open(join(checkpoints_path, checkpoint_name, "cfg.yaml")) as f:
        cfg = yaml.safe_load(f)
    # initialize the data dict for this checkpoint
    metrics_per_run = []
    # loops over each subject
    for subject_id in subjects_in_checkpoint:
        # load the metrics for this subject
        with open(
            join(checkpoints_path, checkpoint_name, subject_id, "metrics.yaml")
        ) as f:
            metrics_per_run.append(yaml.safe_load(f)[0])
    # compute the mean metrics for this checkpoint
    metrics_per_run = pd.DataFrame(metrics_per_run)
    metrics_per_run_mean, metrics_per_run_std = (
        metrics_per_run.mean(),
        metrics_per_run.std(),
    )
    # parses some numbers
    for col in metrics_per_run.columns:
        for df in [metrics_per_run_mean, metrics_per_run_std]:
            if col.startswith("cls_"):
                df[col] = (df[col] * 100).round(3)
            elif col.startswith("num_params"):
                df[col] = (df[col] / 1e3).astype(int)
            elif col.startswith("time_test"):
                df[col] = (df[col] * 1e3).round(3)
    metrics_per_run = (
        metrics_per_run_mean.astype(str) + " ± " + metrics_per_run_std.astype(str)
    )
    metrics_per_run["image_backbone"] = cfg["image_backbone_name"]
    metrics_per_run["landmarks_backbone"] = cfg["landmarks_backbone_name"]
    # append to the metrics list
    metrics.append(metrics_per_run)

# parses the metrics into a DataFrame
metrics = pd.DataFrame(metrics)
metrics = metrics[
    [
        "landmarks_backbone",
        "image_backbone",
        "cls_acc_test",
        "cls_f1_test",
        "cls_prec_test",
        "cls_rec_test",
        "cls_loss_test",
        "num_params_test",
        "time_test",
    ]
]
metrics = metrics.rename(
    columns={
        "image_backbone": "Image backbone",
        "landmarks_backbone": "Landmarks backbone",
        "cls_acc_test": "Accuracy",
        "cls_f1_test": "F1",
        "cls_prec_test": "Precision",
        "cls_rec_test": "Recall",
        "cls_loss_test": "Loss",
        "num_params_test": "# Params (k)",
        "time_test": "Time (ms)",
    }
)
metrics = metrics.sort_values(by=["Landmarks backbone", "Image backbone"])
metrics = metrics.reset_index(drop=True)
print(metrics.to_latex())

\begin{tabular}{llllllllll}
\toprule
 & Landmarks backbone & Image backbone & Accuracy & F1 & Precision & Recall & Loss & # Params (k) & Time (ms) \\
\midrule
0 & linear & clip-b & 81.506 ± 7.848 & 80.954 ± 8.017 & 82.626 ± 7.938 & 81.506 ± 7.848 & 114.761 ± 16.543 & 91012.0 ± 0.0 & 2.526 ± 0.038 \\
1 & linear & convnextv2-b & 83.859 ± 6.415 & 83.26 ± 6.853 & 84.541 ± 7.112 & 83.859 ± 6.415 & 109.614 ± 12.843 & 91773.0 ± 0.0 & 4.653 ± 0.027 \\
2 & linear & convnextv2-t & 83.979 ± 7.173 & 83.384 ± 7.673 & 84.516 ± 7.891 & 83.979 ± 7.173 & 110.759 ± 14.0 & 31422.0 ± 0.0 & 3.041 ± 0.037 \\
3 & linear & dinov2-b & 82.716 ± 7.266 & 81.652 ± 7.569 & 83.426 ± 7.627 & 82.716 ± 7.266 & 112.649 ± 14.693 & 90136.0 ± 0.0 & 2.71 ± 0.015 \\
4 & linear & dinov2-s & 83.24 ± 6.908 & 82.353 ± 7.43 & 83.652 ± 8.062 & 83.24 ± 6.908 & 112.523 ± 14.025 & 24826.0 ± 0.0 & 2.705 ± 0.016 \\
5 & linear & resnet18 & 77.311 ± 8.926 & 76.397 ± 9.01 & 78.673 ± 8.995 & 77.311 ± 8.926 & 123.899 ± 19.582 & 14208.0 ± 0.